# Level 2: Simple Agentic 

This tutorial is designed to give new users an overview of how to use llama-stack's builtin agent framework effectively. It demonstrates basic single-tool usage and provides guidance on switching between a local development environment and a remote kubernetes cluster.

## Overview

This tutorial walks you through how to build your own AI agent who can search the web following these steps:
1. Connecting to a llama-stack server (remote or local).
2. Configuring an agent for tool use.
3. Running the agent.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.2.2`. Lets begin by installing them:


In [1]:
# !pip install llama-stack-client==0.2.2 llama-stack==0.2.2

## Building a Web Search Agent
### 1. Connecting to a llama-stack server.
#### 1.1 Setting Up the Environment
- Import the necessary libraries.
- Defining user variables.
- Setting your ENV variables: Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables
- Connection options
    - **Remote Server**: Set `remote=True` and provide the remote llama-stack endpoint.
    - **Local Server**: Set `remote=False` and provide the local llama-stack endpoint (default port: 8321).
    
    For detailed llama-stack server setup instructions, refer to:
    - [Remote Setup Guide](../../../kubernetes/README.md)
    - [Local Setup Guide](../../../local_setup_guide.md)

In [ ]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client import LlamaStackClient
import os
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint
from dotenv import load_dotenv
load_dotenv()

remote = True # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
stream_output = False # Set to True to stream the output of the agent.
inference_model = "ibm-granite/granite-3.2-8b-instruct"
print(f"Model: {inference_model}")

tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")  # necessary for websearch tool
if remote:
    base_url = os.getenv("REMOTE_BASE_URL")
else:
    base_url = "http://localhost:8321"

Model: ibm-granite/granite-3.2-8b-instruct


#### 1.2 Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.  
- Connect the llama-stack client to the llama-stack server using the base URL.

In [3]:
client = LlamaStackClient(
        base_url=base_url,
        provider_data={"tavily_search_api_key": tavily_search_api_key}
)

### 2. Configuring an agent for tool use.
- **Agent Initialization**: 

- Create an `Agent` instance with the desired model, instructions and tools."

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `builtin::websearch` tool is used, which enables the agent to perform web searches. This tool is essential for retrieving up-to-date information from the web. Internally, it leverages Tavily Search to execute the search queries efficiently.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving information from the web, the agent invokes the `builtin::websearch` tool. The tool interacts with Tavily Search to fetch real-time data, which is then processed and returned to the user in a friendly and helpful tone. This workflow ensures that the agent can handle a wide range of queries effectively.

For more details on the `builtin::websearch` tool and its capabilities, refer to the [Llama-stack tools documentation](https://llama-stack.readthedocs.io/en/latest/building_applications/tools.html#web-search-providers). 

In [4]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)

### 3. Running the agent.
- Define user prompts to interact with the agent.
- Use the agent to create a session and process user queries.
- Display the agent's responses for each query.

In [5]:
user_prompts = [
    "What’s latest in OpenShift?",
    "Search for recent news articles about advancements in quantum computing.",
    "Find coffee shops near Boston Convention and Exhibition Center that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: What’s latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest updates on OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest updates on OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Red Hat unveils OpenShift 4.18: Enhanced security and virtualization ...',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418',
│   │   │   'content': 'This article highlights the latest OpenShift 4.18 innovations and key enhancements. For a comprehensive list of updates and improvements, refer to the OpenShift 4.18 Release Notes. Enhance network flexibility with User Defined Networks and Border Gateway Protocol for pods and virtual machines',
│   │   │   'score': 0.85567063,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Experience the many enhancements in OpenShift 4.18',
│   │   │   'url': 'https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418',
│   │   │   'content': "For an in-depth review of all of the latest features, including updates for plug-ins and templates, refer to What's new in Red Hat Developer Hub 1.4. Red Hat OpenShift Virtualization Engine It is optimized for bare metal operating in a limited space and supports 128 cores per bare-metal socket pair.",
│   │   │   'score': 0.78963995,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Red Hat OpenShift 4.17 is now generally available. Based on Kubernetes 1.30 and CRI-O 1.30, OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.These additions further accelerate innovation with OpenShift without compromising on',
│   │   │   'score': 0.767429,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift Container Platform Life Cycle Policy',
│   │   │   'url': 'https://access.redhat.com/support/policy/updates/openshift/',
│   │   │   'content': 'This life cycle page refers only to the latest major version of Red Hat OpenShift. ... and Important impact security updates and urgent-priority bug fixes for a predefined set of minor releases of Red Hat OpenShift. Extended Update Support - Long Life Add-on - Additional Term 1 enables customers to remain with the same minor release of Red Hat',
│   │   │   'score': 0.6466616,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new in Red Hat OpenShift",
│   │   │   'url': 'https://www.redhat.com/en/whats-new-red-hat-openshift',
│   │   │   'content': 'Red Hat AI A portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud.; Red Hat Enterprise Linux A flexible, stable operating system to support hybrid cloud innovation.; Red Hat OpenShift A container platform to build, modernize, and deploy applications at scale.; Red Hat Ansible Automation Platform A foundation for implementing enterprise-wide automation.',
│   │   │   'score': 0.31019315,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are the latest updates on OpenShift:

1. **Red Hat unveils OpenShift 4.18: Enhanced security and virtualization**
   - This version introduces improvements in network flexibility with User Defined Networks and Border Gateway Protocol for pods and virtual machines.
   - For a detailed list of updates and improvements, check out the [OpenShift 4.18 Release Notes](https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418).

2. **Experience the many enhancements in OpenShift 4.18**
   - This article provides an overview of the latest features, including updates for plug-ins and templates.
   - For a comprehensive review, refer to [What's new in Red Hat Developer Hub 1.4](https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418).

3. **Red Hat OpenShift 4.17: What you need to know**
   - OpenShift 4.17, based on Kubernetes 1.30 and CRI-O 1.30, offers expanded control plane 

{
│   'query': 'recent news articles about advancements in quantum computing',
│   'top_k': [
│   │   {
│   │   │   'title': 'Quantum Computing News -- ScienceDaily',
│   │   │   'url': 'https://www.sciencedaily.com/news/matter_energy/quantum_computing/',
│   │   │   'content': 'Quantum Computing News. Read the latest about the development of quantum computers.',
│   │   │   'score': 0.6897452,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Quantum computing | MIT News | Massachusetts Institute of Technology',
│   │   │   'url': 'https://news.mit.edu/topic/quantum-computing',
│   │   │   'content': 'Modular, scalable hardware architecture for a quantum computer A new quantum-system-on-chip enables the efficient control of a large array of qubits, moving toward practical quantum computing.',
│   │   │   'score': 0.5568099,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'New quantum phase observed for first time, with major implications for ...',
│   │   │   'url': 'https://www.livescience.com/technology/computing/scientists-observe-new-quantum-phase-that-could-have-major-implications-for-quantum-computing',
│   │   │   'content': 'The exotic quantum phase, predicted over half a century ago, could lead to advances in quantum computing, sensors and communication technology.',
│   │   │   'score': 0.46832922,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'New QPU benchmark will show when quantum computers surpass existing ...',
│   │   │   'url': 'https://www.livescience.com/technology/computing/new-qpu-benchmark-will-show-when-quantum-computers-surpass-existing-computing-capabilities-scientists-say',
│   │   │   'content': 'A new benchmark performed on chips from five different vendors has indicated how we can measure QPU performance as quantum computers become more advanced and useful.',
│   │   │   'score': 0.464392,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Big Tech Has Entered Its Quantum Era With Major Advancements - Business ...',
│   │   │   'url': 'https://www.businessinsider.com/big-tech-officially-entered-quantum-era-major-advancements-2025-4?op=1',
│   │   │   'content': "Major advancements in tech and science are brewing as the industry makes its way through the 'International Year of Quantum.'",
│   │   │   'score': 0.45318982,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some recent news articles about advancements in quantum computing:

1. [Quantum Computing News -- ScienceDaily](https://www.sciencedaily.com/news/matter_energy/quantum_computing/)
   - Content: Read the latest about the development of quantum computers.

2. [Quantum computing | MIT News | Massachusetts Institute of Technology](https://news.mit.edu/topic/quantum-computing)
   - Content: A new quantum-system-on-chip enables the efficient control of a large array of qubits, moving toward practical quantum computing.

3. [New quantum phase observed for first time, with major implications for ...](https://www.livescience.com/technology/computing/scientists-observe-new-quantum-phase-that-could-have-major-implications-for-quantum-computing)
   - Content: The exotic quantum phase, predicted over half a century ago, could lead to advances in quantum computing, sensors and communication technology.

4. [New QPU benchmark w

{
│   'query': 'coffee shops near Boston Convention and Exhibition Center open now with Wi-Fi',
│   'top_k': [
│   │   {
│   │   │   'title': 'Best Coffee near Boston Convention and Exhibition Center in ... - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston',
│   │   │   'content': 'Reviews on Coffee near Boston Convention and Exhibition Center - Cocorico Boulangerie, Blue Bottle Coffee, Flour Bakery + Café, Tatte Bakery & Cafe, Caffè Nero, Tradesman Coffee Shop & Lounge, Seaport Bakery, Gracenote Coffee, Blank Street Coffee, Levain Bakery - Boston Seaport ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor',
│   │   │   'score': 0.92343384,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'The Best 10 Cafes near MBTA - Hynes Convention Center in Boston, MA - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?cflt=cafes&find_near=mbta-hynes-convention-center-boston',
│   │   │   'content': 'Best Cafes near MBTA - Hynes Convention Center - Tatte Bakery & Cafe, Jonquils Cafe & Bakery, Beacon Hill Books & Cafe, Greystone Cafe, Bakery, and Provisions, Say Weekend, Cafe 26, Flour Bakery + Cafe, Madhouse Cafe, 180 Cafe ... Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. ... Black Owned Coffee Shop in Boston',
│   │   │   'score': 0.8488849,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Must-Visit Coffee Shops Near Our Convention Centers (Part II)',
│   │   │   'url': 'https://blog.signatureboston.com/must-visit-coffee-shops-near-our-convention-centers-part-ii',
│   │   │   'content': "Hynes. 176,480 square feet of adaptable exhibit space, auditorium with seating for 4,000+, 38 meeting rooms, and a 24,544-square-foot ballroom, all connected to free Wi-Fi and 3,100+ hotel rooms and 200 shops and restaurants in the heart of Boston's Back Bay.",
│   │   │   'score': 0.6922474,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Restaurants near Boston Convention and Exhibition Center',
│   │   │   'url': 'https://www.restaurantji.com/ma/places/boston-convention-and-exhibition-center/',
│   │   │   'content': 'Your guide to the best restaurants near Boston Convention and Exhibition Center. Explore local flavors and hidden gems with Restauranji. ... Coffee Shops • $ 7AM - 3PM 450 Summer Street, ... Not an open air rooftop restaurant but the view is nice if you are facing away from the convention center. A bit noisy, lots of noise coming from the',
│   │   │   'score': 0.5440511,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Best coffee shop near the convention center for a coffee ... - Reddit',
│   │   │   'url': 'https://www.reddit.com/r/boston/comments/1cljyz6/best_coffee_shop_near_the_convention_center_for_a/',
│   │   │   'content': "For a coffee enthusiast, it's definitely Gracenote, but you're gonna have to walk about 20 min from the convention center. Super nice folks run it and they always have the best indie music playing. If you wanna walk 30 min from the convention center, you could reach George Howell which is also fantastic.",
│   │   │   'score': 0.47475353,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near the Boston Convention and Exhibition Center that are currently open and offer Wi-Fi:

1. **Cocorico Boulangerie** - Reviews on Yelp mention that they are open now, offer delivery and takeout, and have free Wi-Fi. You can find them at [this link](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

2. **Blue Bottle Coffee** - Also listed on Yelp, they are open now, offer delivery and takeout, and have free Wi-Fi. Check out their details [here](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

3. **Flour Bakery + Café** - This place is also mentioned on Yelp. They are open now, offer delivery and takeout, and have free Wi-Fi. You can find more information [here](https://www.yelp.com/search?find_desc=Coffee&find_near=boston-convention-and-exhibition-center-boston).

4. **Tatte Bakery & Ca

## Output Analysis
Here, we can observe that the `builtin::websearch` tool is used to perform a web search. The outputs are displayed in the notebook with color-coded text to help interpret the process:

- **Blue Text**: Represents the user's input or query.
- **Yellow Text**: Displays the LLM's inference response. 
- **Green Text**: Indicates the tool execution process, such as the tool being called and the query being sent to the web search API.

Great! 
We can see that the model returned some relevant and recent information about OpenShift. This was only possible due to its ability to call tools like the web search, demonstrating the agent's capacity to retrieve real-time data effectively.

For the second query, the model fetched the latest news articles, including one from April 2025. This is particularly impressive since the Granite 3.2 8B model, released in February 2025, has a knowledge cutoff date of April 2024. This demonstrates the agent's capability to extend beyond its static knowledge base by dynamically retrieving current information.  

For the third query about nearby coffee shops, the agent effectively used the `builtin::websearch` tool to provide practical, location-specific results with valid coffee shops and links, highlighting its real-world applicability.  

## Key Takeaways
- This tutorial demonstrates how to set up and use a single tool with AI Agent.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with AI Agents and explore its capabilities.

For more advanced use cases, please check our other example jupyter notebooks.